### Text Generation with Pretrained NanoGPT

This notebook shows how to generate text using a pretrained NanoGPT model.

By default:

1. The pretrained model and metadata are loaded from the `baselines/nanogpt/shakespeare-char` folder.
2. Generation starts from a configurable prompt (e.g., "To be, or not to be...").

Loading libraries and setting up the environment

In [2]:
import sys
import os

root_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(root_path)

In [3]:
import torch
import pickle
from contextlib import nullcontext
from baselines.nanogpt.model import GPT, GPTConfig

Configuring the model for sampling

In [4]:
from config import MODEL_PATH, META_PATH

START_PROMPT = "to be, or not to be -that is the question:\n"
NUM_SAMPLES = 1
MAX_NEW_TOKENS = 500
TEMPERATURE = 0.8
TOP_K = 200
SEED = 42
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DTYPE = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16'
COMPILE = False

In [5]:
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

device_type = 'cuda' if DEVICE == 'cuda' else 'cpu'
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[DTYPE]
ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)

/Users/sofianikolenko/Downloads/Projects_25/ADL/adl-bnn-textgen/.venv/lib/python3.14/site-packages/torch/backends/__init__.py:46: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/Context.cpp:85.)
  self.setter(val)


Loading the model and tokenizer

In [6]:
# Loading model

checkpoint = torch.load(MODEL_PATH, map_location='cpu', weights_only=False)

gptconf = GPTConfig(**checkpoint['model_args'])
model = GPT(gptconf)

# Remove unwanted prefixes if any
state_dict = checkpoint['model']
unwanted_prefix = '_orig_mod.'
for k, v in list(state_dict.items()):
    if k.startswith(unwanted_prefix):
        state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
model.load_state_dict(state_dict)
model.eval().to(DEVICE)

if COMPILE:
    model = torch.compile(model)

# Loading tokenizer
with open(META_PATH, 'rb') as f:
    meta = pickle.load(f)

stoi, itos = meta['stoi'], meta['itos']
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])


number of parameters: 10.65M


Example: Shakespeare text generation

In [7]:
START_PROMPT = "to be, or"

In [8]:
# Encoding prompt 
if START_PROMPT.startswith("FILE:"):
    with open(START_PROMPT[5:], 'r', encoding='utf-8') as f:
        START_PROMPT = f.read()

start_ids = encode(START_PROMPT)
x = torch.tensor(start_ids, dtype=torch.long, device=DEVICE)[None, ...]

# Generating text samples
with torch.no_grad():
    for i in range(NUM_SAMPLES):
        y = model.generate(x, max_new_tokens=MAX_NEW_TOKENS, temperature=TEMPERATURE, top_k=TOP_K)
        print(f"--- Sample {i+1} ---")
        print(decode(y[0].tolist()))
        print()

--- Sample 1 ---
to be, or else that breaths my last,
And yet love the other sword to be ashed
My service out of the air morning of the present,
Or to us the roof for't: to swear on us,
Were theyselves to give eat them and admire
To be a conspire to be praised
To stay what we seems of their countrymen?

FRIAR LAURENCE:
Madam, come, sit in presence worthy Castle word;
And says to make me what water I am.
What, lords?

KATHARD III:
Sir, have I not seen thus accused by your handship overy rage.
The beauty years; for one tha

